In [1]:
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.1 requires flatbuffers<2,>=1.12, but you have flatbuffers 23.1.21 which is incompatible.
tensorflow 2.9.1 requires gast<=0.4.0,>=0.2.1, but you have gast 0.5.3 which is incompatible.
tensorflow 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
jupyter-server 1.18.1 requires anyio<4,>=3.1.0, but you have anyio 4.5.2 which is incompatible.


In [3]:
"""https://www.youtube.com/watch?v=P8tOjiYEFqU"""
import pandas as pd
import openai
from openai import OpenAI
import os
import ast
import numpy as np
import pdb
print(openai.__version__)

1.55.3


In [5]:
question="What temperature do I set my home at while away so pipes don't freeze?"
question

"What temperature do I set my home at while away so pipes don't freeze?"

In [6]:
response=client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", 
               "content": question}])
response

ChatCompletion(id='chatcmpl-AfFVEal9mVvnffhxJwNKR34XQ4BH5', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="It is recommended to set your home's temperature to at least 55 degrees Fahrenheit while away to prevent pipes from freezing.", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1734394404, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=24, prompt_tokens=23, total_tokens=47, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [7]:
print(response.choices[0].message.content)

It is recommended to set your home's temperature to at least 55 degrees Fahrenheit while away to prevent pipes from freezing.


In [9]:
response=client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Your are an assistant who is funny answering questions as an poem"},
        {"role": "user", "content": question}
    ])
print(response.choices[0].message.content)

Oh dear friend, don't let pipes freeze, 
Set your thermostat to a cool degrees. 
Around 55 should do the trick, 
Ensuring your home stays nice and slick. 
So when you come back from your trip, 
You won't find a plumbing blip.


In [26]:
"""Adding RAG"""
question="how many service calls were serviced in April of 1949 and were those higher than lover compared to other years? By how much?"
question

'how many service calls were serviced in April of 1949 and were those higher than lover compared to other years? By how much?'

In [27]:
df = pd.read_csv('db1.csv')
df.head()

,Year and Month of service call,Number of service calls
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121


In [28]:
def get_embedding(text, model="text-embedding-3-small"):
                  text=text.replace("\n", " ")
                  return client.embeddings.create(input=[text], model=model).data[0].embedding

In [30]:
text=df.to_string()
print(df.to_string())

    Year and Month of service call  Number of service calls
0                          1949-01                      112
1                          1949-02                      118
2                          1949-03                      132
3                          1949-04                      129
4                          1949-05                      121
5                          1949-06                      135
6                          1949-07                      149
7                          1949-08                      148
8                          1949-09                      136
9                          1949-10                      119
10                         1949-11                      104
11                         1949-12                      118
12                         1950-01                      115
13                         1950-02                      126
14                         1950-03                      141
15                         1950-04      

In [31]:
get_embedding(text)

[0.013647603802382946,
 -0.0021925659384578466,
 0.05029477924108505,
 0.014102524146437645,
 0.0616304911673069,
 0.017749343067407608,
 0.035230208188295364,
 -0.03376849740743637,
 0.009374338202178478,
 0.004504455253481865,
 0.01032146718353033,
 0.004523099400103092,
 -0.00545158376917243,
 0.0025710444897413254,
 0.014042862690985203,
 -0.004556659143418074,
 0.016272718086838722,
 -0.003881736658513546,
 -0.021239550784230232,
 0.05557483434677124,
 0.039525847882032394,
 0.02316363900899887,
 -0.015221180394291878,
 -0.024535857141017914,
 0.0049295444041490555,
 -0.01431879773736,
 -0.030636262148618698,
 -0.03854143247008324,
 0.010776386596262455,
 -0.03693056479096413,
 0.033380698412656784,
 -0.03567767143249512,
 -0.01136554591357708,
 -0.01944969967007637,
 0.026236213743686676,
 -0.011186560615897179,
 0.025326373055577278,
 -0.0003838388656731695,
 -0.01557169295847416,
 -0.002889861585572362,
 0.00812144298106432,
 -0.029577266424894333,
 0.0006558121531270444,
 0.03

In [32]:
response=client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages = [
        {"role": "system", "content": " you are an assistantwho is helping answering questions"},
        {"role": "user", "content": question},
        {"role": "assistant", "content": f"Use this information from website content to answer question: {text}. Please stick to this context when answering questions."}
    ])

In [33]:
print(question)
print(text)

how many service calls were serviced in April of 1949 and were those higher than lover compared to other years? By how much?
    Year and Month of service call  Number of service calls
0                          1949-01                      112
1                          1949-02                      118
2                          1949-03                      132
3                          1949-04                      129
4                          1949-05                      121
5                          1949-06                      135
6                          1949-07                      149
7                          1949-08                      148
8                          1949-09                      136
9                          1949-10                      119
10                         1949-11                      104
11                         1949-12                      118
12                         1950-01                      115
13                         1950-02 

In [34]:
response.choices[0].message.content

'In April 1949, there were 129 service calls serviced. To determine if this number is higher or lower compared to other years, we need to compare it to the number of service calls serviced in the month of April for each year listed. \n\nHere is a comparison of service calls in April for the following years:\n- April 1949: 129\n- April 1950: 135\n- April 1951: 163\n- April 1952: 181\n- April 1953: 235\n- April 1954: 227\n- April 1955: 269\n- April 1956: 313\n- April 1957: 348\n- April 1958: 348\n- April 1959: 396\n- April 1960: 461\n\nFrom the comparison, we can see that the number of service calls serviced in April 1949 was lower than the number of service calls in April for most of the other years. It was higher than 1949, 1950, and 1954. The largest difference was with 1960, where there were 461 service calls in April, significantly more than in 1949.'